In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('output/feature_df.csv', sep=';', low_memory=False)
dataset = data.copy(deep=True)
dataset.head()
dataset.columns
dataset.info()
dataset.drop(columns=['dos_id', 'motif_admission', 'ratio_medecin_patient', 'ratio_interne_patient',
                      'ratio_infirmiere_patient', 'nbre_patient_salle_att_ext', 'nbre_patient_salle_att_interne',
                      'nbre_patient_traite_sans_fin_PEC', 'nbre_patient_entre_finPEC_sortie',
                      'nbre_patient_UHCD', 'UHCD_saturee', 'est_venu_3_derniers_mois',
                      'est_venu_derniers_48h', 'ven_admission', ], inplace=True)
dataset.shape
dataset.head()
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

dataset.info()
# Separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    dataset.drop(['wait_time_ext', 'wait_time_interne', 'wait_time'], axis=1),
    dataset['wait_time'],
    test_size=0.3,
    random_state=0
)
# set up the imputer
# Use most frequent values to impute categorical data
mode_imputer = SimpleImputer(strategy='most_frequent')
# fit the imputer
mode_imputer.fit(X_train)
# transform the data
X_train = mode_imputer.transform(X_train)
X_train.shape
X_train = pd.DataFrame(X_train)
X_train.columns = dataset.columns[:-3]
X_train.info()
y_train = y_train.reset_index(drop=True)
y_train.head()
X_test = mode_imputer.transform(X_test)
X_test = pd.DataFrame(X_test)
X_test.columns = dataset.columns[:-3]
X_test.info()
dataset.info()

#Begin test code <temporal>
df = X_train.copy(deep=True)
df.head()
df["wait_time"] = pd.DataFrame(y_train)["wait_time"]
df = df[['motif_code', 'wait_time']]
df.head()
df.sort_values(by=["motif_code"], inplace=True)
df.head()
df.info()
df_plot = df.groupby(by=["motif_code"]).mean()
df_plot["motif_code"] = df_plot.index
df_plot.head()
#Dict of motif_code with mean wait time
reg = dict(zip(df_plot.motif_code, df_plot.wait_time))
reg
reg['O20.9']
df_plot["std"] = df.groupby(by=["motif_code"]).std()
df_plot.head()
#df_plot["motif_code"] = df_plot.index
df_plot.columns
df_plot["wait_time"].plot()
df_plot["std"].plot()
import matplotlib.pyplot as plt

# plot the dataframe
df_plot.plot(y=["wait_time", "std"], kind="bar", figsize=(39, 28))

# print bar graph
plt.show()
# END tempo code


#Features encoding
from category_encoders import TargetEncoder

# set up the encoder
encoder = TargetEncoder(cols=['motif_code'])

# fit the encoder
encoder.fit(X_train, y_train)
# transform the data
train_t = encoder.transform(X_train)
test_t = encoder.transform(X_test)
train_t
X_train.head()
dataset["motif_code"].value_counts().sort_values()
import matplotlib.pyplot as plt

dataset["motif_code"].value_counts().sort_values().plot()
